In [1]:
from bs4 import BeautifulSoup as BS
import requests
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import datetime
# from pandas.io.common import FileNotFoundError, EmptyDataError

In [2]:
city_from = 'BLR'
city_to = 'DEL'
date_start = '08/04/2021'
date_end = '13/04/2021'

url = f'https://www.makemytrip.com/flight/search?itinerary={city_from}-{city_to}-{date_start}_{city_to}-{city_from}-{date_end}&tripType=R&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng'

In [3]:
url

'https://www.makemytrip.com/flight/search?itinerary=BLR-DEL-08/04/2021_DEL-BLR-13/04/2021&tripType=R&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng'

## By using only SELENIUM:

In [4]:
def click_nonStop(driver):
    Ob1 = driver.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[3]/div/div[1]/div/div[2]/div/div[1]/div/label[1]/div/span[1]/span')
    Ob2 = driver.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[3]/div/div[1]/div/div[3]/div/div[1]/div/label[1]/div/span[1]/span')
    Ob1.click()
    Ob2.click()

In [5]:
def listings(driver):
    listing_left = driver.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[3]/div/div[2]/div/div/div/div[1]/div[2]')
    listing_right = driver.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[3]/div/div[2]/div/div/div/div[2]/div[2]')

    listing_left_list = listing_left.find_elements_by_class_name('splitViewListing')
    listing_right_list = listing_right.find_elements_by_class_name('splitViewListing')
    
    for i, el in enumerate(listing_left_list):
        listing_left_list[i] = el.text
    for i, el in enumerate(listing_right_list):
        listing_right_list[i] = el.text
    
    return listing_left_list, listing_right_list

In [6]:
def merge_dicts(dict_list):
    '''
    Merges a list of dictionaries with same keys
    '''
    output = dict.fromkeys(dict_list[0],[])
    for key in output.keys():
        output[key] = [d[key] for d in dict_list]
    return output

In [22]:
def info_df(listing_info, city_from, city_to):
    Left_info = []
    Right_info = []
    ruppee_symbol = '₹'
    for el_left, el_right in zip(top_listingLeft, top_listingRight):

        timings_left = el_left.split(ruppee_symbol)[0].strip().split('\n')
        cost_left = int(el_left.split(ruppee_symbol)[1].strip().replace(',',''))
        Left_info.append([timings_left[0], timings_left[1], timings_left[3], timings_left[5], cost_left])

        timings_right = el_right.split(ruppee_symbol)[0].strip().split('\n')
        cost_right = int(el_right.split(ruppee_symbol)[1].strip().replace(',',''))
        Right_info.append([timings_right[0], timings_right[1], timings_right[3], timings_right[5], cost_right])
        
    Info = np.concatenate((np.array(Left_info), np.array(Right_info)), axis=1)
    header = [f'Airline ({city_from})', f'Departure ({city_from})', f'Duration ({city_from})', f'Arrival ({city_from})', f'Cost ({city_from})',
              f'Airline ({city_to})', f'Departure ({city_to})', f'Duration ({city_to})', f'Arrival ({city_to})', f'Cost ({city_to})']

    df = pd.DataFrame(data=Info, columns=header)
    return df

In [23]:
def update_info(prev_info, current_info):
    current_update = prev_info.append(current_info)
    return current_update

In [4]:
start = time.time()
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)
print('Time Elapsed in Fetching from site: ', time.time()-start)

Time Elapsed in Fetching from site:  26.504699230194092


In [ ]:
panes = driver.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[2]/div/div[2]')

In [11]:
click_nonStop(driver)
listing_left, listing_right = listings(driver)
driver.quit()

In [12]:
n = int(input('Number of listings:'))
top_listingLeft = listing_left[:n]
top_listingRight = listing_right[:n]

Number of listings: 5


In [26]:
current_info = info_df((top_listingLeft, top_listingRight), city_from, city_to)
current_info.insert(loc=0, column='Date Time', value=np.array(datetime.datetime.now().strftime('%d/%m/%y, %H:%M:%S')))

In [27]:
current_info

,Date Time,Airline (BLR),Departure (BLR),Duration (BLR),Arrival (BLR),Cost (BLR),Airline (DEL),Departure (DEL),Duration (DEL),Arrival (DEL),Cost (DEL)
0,"14/03/21, 11:32:40",IndiGo,12:20,02 h 40 m,15:00,5656,IndiGo,05:15,02 h 40 m,07:55,5607
1,"14/03/21, 11:32:40",Spicejet,05:55,02 h 45 m,08:40,5656,IndiGo,02:25,02 h 45 m,05:10,5607
2,"14/03/21, 11:32:40",Spicejet,09:40,02 h 45 m,12:25,5656,IndiGo,07:20,02 h 45 m,10:05,5607
3,"14/03/21, 11:32:40",AirAsia,11:10,02 h 45 m,13:55,5656,IndiGo,19:55,02 h 45 m,22:40,5607
4,"14/03/21, 11:32:40",IndiGo,19:25,02 h 45 m,22:10,5656,IndiGo,23:15,02 h 45 m,02:00,5607


In [28]:
filename = f'{city_from}_{city_to}_Tickets_MMT.csv'
try:
    prev_info = pd.read_csv(filename)
except:
    current_info.to_csv(filename, index=False)
else:
    current_update = update_info(prev_info, current_info)
    current_update.to_csv(filename, index=False)

In [3]:
now = datetime.datetime.now().strftime('%d/%m/%y, %H:%M:%S')
print(now)

13/03/21, 16:09:03


## Using SELENIUM and BEAUTIFUL SOUP

In [ ]:
driver = webdriver.Chrome()
driver.get(url)
# time.sleep(3)

In [10]:
soup = BS(driver.page_source, 'lxml')
driver.quit()

In [32]:
print(soup.body.prettify())

<body class="">
 <div id="root">
  <div data-reactroot="">
   <div class="commonHeader">
    <div class="singleHeader" data-cy="landingContainer">
     <div class="headerOuter">
      <div class="chHeaderWrapper">
       <div class="chHeaderContainer">
        <span class="logoContainer">
         <a class="chMmtLogo" href="https://www.makemytrip.com/">
          <img alt="MMT's LOGO" src="//imgak.mmtcdn.com/pwa_v3/pwa_hotel_assets/header/logo@2x.png"/>
         </a>
        </span>
        <nav class="">
         <ul class="makeFlex font12">
          <li class="menu_Flights" data-cy="menu_Flights">
           <a class="active makeFlex hrtlCenter column" href="https://www.makemytrip.com/flights/">
            <span class="chNavIcon appendBottom2 chSprite chFlights active">
            </span>
            <span class="chNavText darkGreyText">
             Flights
            </span>
           </a>
          </li>
          <li class="menu_Hotels" data-cy="menu_Hotels">
           <a c

In [11]:
import params

In [12]:
params.date_start

AttributeError: module 'params' has no attribute 'date_start'